# Individual Report \- ACS Housing and Economics

\(Because the social data is not available for all states, I wiil just focus on the other two\)


In [1]:
import pandas as pd
import numpy as np

In [2]:
housing = pd.read_csv("./data/ACSDP1Y2022.DP04-Data.csv", low_memory=False)
economics = pd.read_csv("./data/ACSDP1Y2022.DP03-Data.csv", low_memory=False)
housing_labels = pd.read_csv("data/ACSDP1Y2022.DP03-Column-Metadata.csv")
economics_labels = pd.read_csv("data/ACSDP1Y2022.DP04-Column-Metadata.csv")

In [3]:
housing.head()

,GEO_ID,NAME,DP04_0001E,DP04_0001M,DP04_0001MA,DP04_0001EA,DP04_0002E,DP04_0002EA,DP04_0002M,DP04_0002MA,...,DP04_0141PMA,DP04_0142PE,DP04_0142PEA,DP04_0142PM,DP04_0142PMA,DP04_0143PE,DP04_0143PEA,DP04_0143PM,DP04_0143PMA,Unnamed: 1146
0,Geography,Geographic Area Name,Estimate!!HOUSING OCCUPANCY!!Total housing units,Margin of Error!!HOUSING OCCUPANCY!!Total hous...,Annotation of Margin of Error!!HOUSING OCCUPAN...,Annotation of Estimate!!HOUSING OCCUPANCY!!Tot...,Estimate!!HOUSING OCCUPANCY!!Total housing uni...,Annotation of Estimate!!HOUSING OCCUPANCY!!Tot...,Margin of Error!!HOUSING OCCUPANCY!!Total hous...,Annotation of Margin of Error!!HOUSING OCCUPAN...,...,Annotation of Percent Margin of Error!!GROSS R...,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHO...,Annotation of Percent!!GROSS RENT AS A PERCENT...,Percent Margin of Error!!GROSS RENT AS A PERCE...,Annotation of Percent Margin of Error!!GROSS R...,Percent!!GROSS RENT AS A PERCENTAGE OF HOUSEHO...,Annotation of Percent!!GROSS RENT AS A PERCENT...,Percent Margin of Error!!GROSS RENT AS A PERCE...,Annotation of Percent Margin of Error!!GROSS R...,NaN
1,0500000US01003,"Baldwin County, Alabama",132299,185,NaN,NaN,98854,NaN,3781,NaN,...,NaN,46.3,NaN,9.2,NaN,(X),(X),(X),(X),NaN
2,0500000US01015,"Calhoun County, Alabama",53408,87,NaN,NaN,45701,NaN,1562,NaN,...,NaN,34.9,NaN,8.8,NaN,(X),(X),(X),(X),NaN
3,0500000US01043,"Cullman County, Alabama",39893,40,NaN,NaN,35966,NaN,1274,NaN,...,NaN,39.7,NaN,14.3,NaN,(X),(X),(X),(X),NaN
4,0500000US01049,"DeKalb County, Alabama",31022,39,NaN,NaN,26459,NaN,1114,NaN,...,NaN,32.4,NaN,12.6,NaN,(X),(X),(X),(X),NaN


We can see the most import column is the GEO\_ID which seems to to have the same last 5 digits as the County FIPS form the COVID dataset. We can use this to join. The next column is the Geographic Name with the state. Then every three columns are called "DP04\_xxxx" with either

- E for Estimate 
- M for Margin of Error
- EA for Annotation of Estimate
- MA for Annotation of Margin of Error

The datatype is depent on the specific column. For detailed descriptions of the columns, we can refer to the label datasets, which contain the abbreviations and the complete labels. Note that all column can contain NaN when data is missing.

The same is true for the economic set:


In [4]:
economics.head()

,GEO_ID,NAME,DP03_0001E,DP03_0001M,DP03_0001MA,DP03_0001EA,DP03_0002E,DP03_0002M,DP03_0002MA,DP03_0002EA,...,DP03_0135PMA,DP03_0136PE,DP03_0136PEA,DP03_0136PM,DP03_0136PMA,DP03_0137PE,DP03_0137PM,DP03_0137PMA,DP03_0137PEA,Unnamed: 1098
0,Geography,Geographic Area Name,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Annotation of Margin of Error!!EMPLOYMENT STAT...,Annotation of Estimate!!EMPLOYMENT STATUS!!Pop...,Estimate!!EMPLOYMENT STATUS!!Population 16 yea...,Margin of Error!!EMPLOYMENT STATUS!!Population...,Annotation of Margin of Error!!EMPLOYMENT STAT...,Annotation of Estimate!!EMPLOYMENT STATUS!!Pop...,...,Annotation of Percent Margin of Error!!PERCENT...,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHO...,Annotation of Percent!!PERCENTAGE OF FAMILIES ...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Annotation of Percent Margin of Error!!PERCENT...,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHO...,Percent Margin of Error!!PERCENTAGE OF FAMILIE...,Annotation of Percent Margin of Error!!PERCENT...,Annotation of Percent!!PERCENTAGE OF FAMILIES ...,NaN
1,0500000US01003,"Baldwin County, Alabama",201504,1225,NaN,NaN,117259,3370,NaN,NaN,...,NaN,11.0,NaN,2.5,NaN,25.9,5.3,NaN,NaN,NaN
2,0500000US01015,"Calhoun County, Alabama",94762,667,NaN,NaN,53438,2853,NaN,NaN,...,NaN,14.5,NaN,3.6,NaN,33.2,5.9,NaN,NaN,NaN
3,0500000US01043,"Cullman County, Alabama",72822,501,NaN,NaN,43353,2012,NaN,NaN,...,NaN,15.1,NaN,3.9,NaN,31.9,7.4,NaN,NaN,NaN
4,0500000US01049,"DeKalb County, Alabama",56340,610,NaN,NaN,32603,1753,NaN,NaN,...,NaN,15.9,NaN,4.8,NaN,38.3,6.4,NaN,NaN,NaN


We can merge the datasets based on the 5 digit county codes. Using the housing and economics data, we can show that the spread of COVID waws not evenly spread, and that housing and econimcs have an effect on how its spreads. The idea is to check which identifyers have a strong correlation with the spread of covid, and which don't
